In [1]:
#Description: 

import os
import cv2
import math
import numpy as np 
from numpy import savetxt, genfromtxt

import silhouette_extract
import projection


#Setting the path
data_dir = os.path.expanduser('~')
data_dir = data_dir + "/Downloads/DataSets/beethoven_data/images/"

#Dimensions of the initial canvas of voxels           
step_size = 2 #0.42
xmin = -8
xmax = 5
ymin = -8
ymax = 8
zmin = -5
zmax = 16 #17.5


total_voxels=int((xmax-xmin)/step_size)*int((ymax-ymin)/step_size)*int((zmax-zmin)/step_size)

#Image dimensions
height = 768
width = 1024

#Initial canvas of voxels defined as a three dimentional array of true booleans
obj = np.ones((int((xmax-xmin)/step_size), int((ymax-ymin)/step_size), int((zmax-zmin)/step_size)), dtype = bool)


out_cnt = 0
carve_cnt = 0
image_pixel_mark_situ = {}
for file in os.listdir(data_dir):
    #Initializing the pixel marking matrix to account for voxels occlusion from some camera views
    pixel_map = np.zeros((height, width), dtype = np.int8)
    image_pixel_mark_situ[file] = pixel_map
#Method to check the voxel consistency using only the background constraint
def background_consistency_check(i, j, k):
    global out_cnt, image_pixel_mark_situ
    consist = True
    voxel_cord = np.array([[xmin + i*step_size], [ymin + j*step_size],[zmin + k*step_size], [1]])

    for file in os.listdir(data_dir):
        #Reading each image's segmentation and projection matrix    
        img_mask = silhouette_extract.get_mask(file)
        P = projection.projectionMatrix(file)
        #Projecting the voxel on the 2D image
        pix_cord = P.dot(voxel_cord)
        pix_cord = pix_cord / pix_cord[2,0] # division by the last vector's element of the
                                                        # homogeneous representation to get the pixel's coordinates in the
                                                        #extracted image

        #if the voxel is projected outside the 2D image boundaries, this means that the voxel is not in the camera's visual field
        #and the info provided by the correspondent 2D image cannot be used
        if pix_cord[1,0] >= height or pix_cord[0,0] < 0 or pix_cord[0,0] >= width or pix_cord[1,0] < 0:
            out_cnt = out_cnt+1
            continue    
        
        if image_pixel_mark_situ[file][int(pix_cord[1,0]), int(pix_cord[0,0])] == 0:
            # Pixels with intensity values 0 (black) denote object occupancy
            if img_mask[int(pix_cord[1,0]), int(pix_cord[0,0]), 0] > 0 or img_mask[int(pix_cord[1,0]), int(pix_cord[0,0]), 1] > 0 or img_mask[int(pix_cord[1,0]), int(pix_cord[0,0]), 2] > 0:
                #If the pixel is part of the background (i.e. not black or white in this case), it will be carved out
                consist = False
                break
            
    if not consist:
        return False
    else:
        
        # area block. This idea is implemented to avoid the extraction and the inclusion in the 
        # calculation of the projected pixel's color from an image where the correspondent voxel cannot be seen
        # in the first place (i.e. the correspondent voxel is hidden by another voxel that was previously proven
        # to be consistent)
        voxel_cord1 = np.array([[xmin + i*step_size+step_size], [ymin + j*step_size+step_size],[zmin + k*step_size+step_size], [1]])
        for file in os.listdir(data_dir):
            P = projection.projectionMatrix(file)
            pix_cord = P.dot(voxel_cord)
            pix_cord = pix_cord / pix_cord[2,0]
            if pix_cord[1,0] >= height or pix_cord[0,0] < 0 or pix_cord[0,0] >= width or pix_cord[1,0] < 0:
                continue 
            if image_pixel_mark_situ[file][int(pix_cord[1,0]), int(pix_cord[0,0])] == 0:
               pix_cord1 = P.dot(voxel_cord1)
               pix_cord1 = pix_cord1 / pix_cord1[2,0]
               pix_length = int(np.linalg.norm(pix_cord-pix_cord1))
        
               for m in range(int(pix_cord[1,0]-pix_length/2),int(pix_cord[1,0] + pix_length/2)):
                    for n in range(int(pix_cord[0,0]-pix_length/2),int(pix_cord[0,0] + pix_length/2)):
                        image_pixel_mark_situ[file][m,n] = 100 #any other non-zero number can be placed here
        return True        

if __name__ == "__main__":
    voxel_count = 0
    eval_list = []
    # carving out the visual hull
    for i in range(obj.shape[0]):
        for j in range(obj.shape[1]):
            for k in range(obj.shape[2]):
                voxel_count +=1
                print("processed "+ str(voxel_count/total_voxels*100) + "% of voxels")
                if not background_consistency_check(i, j, k):
                    obj[i,j,k] = 0
                    carve_cnt = carve_cnt+1  
    #Saving the output 3D shape that was carved from the initial canvas of voxels as a .txt file
    print(obj.size, obj.size-carve_cnt, out_cnt)
    final_file = open("./visual_hull_result.txt", "w")
    head_line = "{} {} {} {} {} {} {} {}\n" # step_size, xmin, xmax, ymin, ymax, zmin, zmax, is_colored version
    voxel_elem = "{} "
    final_file.write(head_line.format(step_size, xmin, xmax, ymin, ymax, zmin, zmax, 0))
            
    for i in range(obj.shape[0]):
        for j in range(obj.shape[1]):
            for k in range(obj.shape[2]):
                final_file.write(voxel_elem.format(int(obj[i,j,k])))
                eval_list.append(int(obj[i,j,k]))  

    final_file.close()
    eval_array = np.array(eval_list)
    savetxt('background_only.csv',eval_array,delimiter=',')                                   
# end of the visual hull build



processed 0.20833333333333334% of voxels
processed 0.4166666666666667% of voxels
processed 0.625% of voxels
processed 0.8333333333333334% of voxels
processed 1.0416666666666665% of voxels
processed 1.25% of voxels
processed 1.4583333333333333% of voxels
processed 1.6666666666666667% of voxels
processed 1.875% of voxels
processed 2.083333333333333% of voxels
processed 2.2916666666666665% of voxels
processed 2.5% of voxels
processed 2.7083333333333335% of voxels
processed 2.9166666666666665% of voxels
processed 3.125% of voxels
processed 3.3333333333333335% of voxels
processed 3.5416666666666665% of voxels
processed 3.75% of voxels
processed 3.958333333333333% of voxels
processed 4.166666666666666% of voxels
processed 4.375% of voxels
processed 4.583333333333333% of voxels
processed 4.791666666666667% of voxels
processed 5.0% of voxels
processed 5.208333333333334% of voxels
processed 5.416666666666667% of voxels
processed 5.625% of voxels
processed 5.833333333333333% of voxels
processed 